In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import numpy as np
import pandas as pd

In [3]:
train_df=pd.read_csv('/content/gdrive/MyDrive/ECGData/mitbih_train.csv',header=None)
test_df=pd.read_csv('/content/gdrive/MyDrive/ECGData/mitbih_test.csv',header=None)

In [4]:
train_df[187]=train_df[187].astype(int)
distribution=train_df[187].value_counts()
print(distribution)

0    72471
4     6431
2     5788
1     2223
3      641
Name: 187, dtype: int64


In [5]:
test_df[187]=test_df[187].astype(int)
distribution=test_df[187].value_counts()
print(distribution)

0    18118
4     1608
2     1448
1      556
3      162
Name: 187, dtype: int64


In [6]:
type(distribution)

pandas.core.series.Series

In [7]:
from sklearn.utils import resample
count1=10000
newtraindf=(train_df[train_df[187]==0]).sample(n=count1)
for i in range(1,5):
  df_i=train_df[train_df[187]==i]
  df_x=resample(df_i,replace=True,n_samples=count1)
  newtraindf=pd.concat([newtraindf,df_x])

newtestdf=pd.DataFrame()
newvaldf=pd.DataFrame()

for i in range(0,5):
  shuffled_df=test_df[test_df[187]==i].sample(frac=1, random_state=42)
  split_index = len(shuffled_df) // 2
  df_half1 = shuffled_df.iloc[:split_index, :]
  df_half2 = shuffled_df.iloc[split_index:, :]
  newtestdf=pd.concat([newtestdf,df_half1])
  newvaldf=pd.concat([newvaldf,df_half1])

In [8]:
def standardize(train,test,final_test):

    mean = np.mean(train, axis=0)
    std = np.std(train, axis=0)+0.000001

    X_train = (train - mean) / std
    X_test = (test-mean)/std
    X_final_test = (final_test-mean)/std
    return X_train,X_test,X_final_test

In [9]:

# Access the datasets in the HDF5 file
train_1_data = newtraindf[newtraindf.columns[:187]].values
train_1_labels = newtraindf[newtraindf.columns[187:188]].values
y_train = train_1_labels.reshape(train_1_labels.shape[0])
train_1_data = train_1_data.reshape((train_1_data.shape[0],1,train_1_data.shape[1]))
train_1_data = train_1_data.astype(float)


test_1_data = newvaldf[newvaldf.columns[:187]].values
test_1_labels = newvaldf[newvaldf.columns[187:188]].values
y_test = test_1_labels.reshape(test_1_labels.shape[0])
test_1_data = test_1_data.reshape((test_1_data.shape[0],1,test_1_data.shape[1]))
test_1_data = test_1_data.astype(float)


final_test_1_data = newtestdf[newtestdf.columns[:187]].values
final_test_1_labels = newtestdf[newtestdf.columns[187:188]].values
y_final_test = final_test_1_labels.reshape(final_test_1_labels.shape[0])
final_test_1_data = final_test_1_data.reshape((final_test_1_data.shape[0],1,final_test_1_data.shape[1]))
final_test_1_data = final_test_1_data.astype(float)

X_train,X_test,X_final_test=standardize(train_1_data,test_1_data,final_test_1_data)

In [10]:
X_train.shape

(50000, 1, 187)

In [11]:
y_train.shape

(50000,)

In [12]:
import torch

In [13]:
# Convert the NumPy arrays to PyTorch tensors
train_loaded_data = torch.from_numpy(X_train)
train_loaded_labels = torch.from_numpy(y_train)

test_loaded_data = torch.from_numpy(X_test)
test_loaded_labels = torch.from_numpy(y_test)

final_test_loaded_data = torch.from_numpy(X_final_test)
final_test_loaded_labels = torch.from_numpy(y_final_test)

In [14]:
from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import LogSoftmax
from torch import flatten
import torch
import torch.nn as nn
import torch.optim as optim

In [15]:
X_train.shape

(50000, 1, 187)

In [16]:
class LSTMModel(nn.Module):
    def __init__(self, input_size=187, hidden_size=64, num_layers=2, num_classes=5):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

        # Fully connected layer
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # Forward pass through LSTM layer
        out, _ = self.lstm(x, (h0, c0))

        # Take the output from the last time step
        out = out[:, -1, :]

        # Forward pass through the fully connected layer
        out = self.fc(out)

        return out

In [17]:
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F

In [18]:
batch_size = 500  # Adjust as needed
train_dataset = TensorDataset(train_loaded_data, train_loaded_labels)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(test_loaded_data, test_loaded_labels)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

final_test_dataset = TensorDataset(final_test_loaded_data, final_test_loaded_labels)
final_test_loader = DataLoader(final_test_dataset, batch_size=batch_size, shuffle=False)

In [19]:
def getAccuracy(neuralmodel,loader):
    neuralmodel.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs = inputs.float()
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = neuralmodel(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate test accuracy
    test_accuracy = correct / total
    return test_accuracy,(f'{test_accuracy * 100:.2f}%')

In [20]:
plotdata=[]
num_epochs = 101
cnn_model = LSTMModel()
criterion = nn.CrossEntropyLoss()
learning_rate = 0.01
# cnn_model.train()
# optimizer = optim.SGD(cnn_model.parameters(),lr=10)
optimizer = optim.SGD(cnn_model.parameters(), lr=learning_rate,momentum=0.9)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cnn_model.to(device)
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        # print(inputs.shape)
        inputs, labels = inputs.to(device), labels.to(device)
        inputs = inputs.float()
        outputs = cnn_model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    if (epoch) % 10 == 0:
        # print(outputs.sum())
        # print(loss.numpy())


        train_val,trainacc=getAccuracy(cnn_model,train_loader)
        test_val,testacc=getAccuracy(cnn_model,test_loader)
        plotdata.append([epoch+1,train_val,test_val])
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
        print("Train Acc ",trainacc," Test Acc ",testacc)

        cnn_model.train()
print("Training complete!")

Epoch [1/101], Loss: 1.5864
Train Acc  37.57%  Test Acc  12.95%
Epoch [11/101], Loss: 0.5566
Train Acc  78.86%  Test Acc  59.03%
Epoch [21/101], Loss: 0.3156
Train Acc  88.87%  Test Acc  84.57%
Epoch [31/101], Loss: 0.2009
Train Acc  92.34%  Test Acc  87.84%
Epoch [41/101], Loss: 0.1481
Train Acc  94.88%  Test Acc  89.55%
Epoch [51/101], Loss: 0.1321
Train Acc  96.28%  Test Acc  90.91%
Epoch [61/101], Loss: 0.0791
Train Acc  97.13%  Test Acc  92.23%
Epoch [71/101], Loss: 0.1032
Train Acc  97.84%  Test Acc  92.79%
Epoch [81/101], Loss: 0.0460
Train Acc  98.36%  Test Acc  92.97%
Epoch [91/101], Loss: 0.0594
Train Acc  98.69%  Test Acc  93.34%
Epoch [101/101], Loss: 0.0441
Train Acc  99.06%  Test Acc  93.86%
Training complete!


In [21]:
plotdf = pd.DataFrame(columns=["Epoch","Train Acc","Test Acc"])

for dat in plotdata:
  plotdf.loc[plotdf.shape[0]]=dat

In [22]:
plotdf.to_csv("/content/gdrive/MyDrive/ECGData/PlotData/lstm_plotdata.csv",index=False)

In [23]:
torch.save(cnn_model.state_dict(), "/content/gdrive/MyDrive/ECGData/Models/LSTM_model.pth")

In [24]:
cnn_model.eval()
correct = 0
total = 0

total_labels=[]
total_predicted=[]
with torch.no_grad():
    for inputs, labels in final_test_loader:
        inputs = inputs.float()
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = cnn_model(inputs)
        total_labels = total_labels +labels.tolist()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        total_predicted = total_predicted+predicted.tolist()
        correct += (predicted == labels).sum().item()

# Calculate test accuracy
test_accuracy = correct / total
print(f'{test_accuracy * 100:.2f}%')

93.86%


In [25]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [26]:
y_true_np = np.array(total_labels)
y_pred_np = np.array(total_predicted)
# Calculate precision, recall, and F1 score
precision = precision_score(y_true_np, y_pred_np, average='weighted')
recall = recall_score(y_true_np, y_pred_np, average='weighted')
f1 = f1_score(y_true_np, y_pred_np, average='weighted')
accuracy = accuracy_score(y_true_np, y_pred_np)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Accuracy:", accuracy)

Precision: 0.9573660759022167
Recall: 0.938607710579207
F1 Score: 0.945263186096288
Accuracy: 0.938607710579207


In [27]:
metrics_df = pd.DataFrame(columns=["Metric","Value"])
metrics_df.loc[0]=["Accuracy",accuracy]
metrics_df.loc[1]=["Precision",precision]
metrics_df.loc[2]=["Recall",recall]
metrics_df.loc[3]=["F1 Score",f1]
metrics_df.to_csv("/content/gdrive/MyDrive/ECGData/Metrics/LSTM_metrics.csv",index=False)